In [43]:
import json
from pprint import pprint

with open('output/data_20161009_1142.json') as data_file:    
    data = json.load(data_file)

data

[{'issue_area': 'Environment',
  'issue_path': '/category/3/',
  'question': 'World population continues to grow exponentially, so what should we be doing?',
  'question_path': '/questions/12143/vote/',
  'submission_date': '2016-10-04T17:59:00',
  'submitter_location': 'CA',
  'submitters': 'Lee M.',
  'vote_count': '27'},
 {'issue_area': 'Foreign Policy & Military',
  'issue_path': '/category/5/',
  'question': 'How many innocent civilians will be killed by drones during your administration?',
  'question_path': '/questions/2849/vote/',
  'submission_date': '2016-09-27T17:12:00',
  'submitter_location': 'NJ',
  'submitters': 'Jersey  B.',
  'vote_count': '27'},
 {'issue_area': 'Environment',
  'issue_path': '/category/6/',
  'question': 'Will you support rebuilding our water infrastructure? ',
  'question_path': '/questions/12504/vote/',
  'submission_date': '2016-10-05T03:02:00',
  'submitter_location': 'MN',
  'submitters': 'Celine  S.',
  'vote_count': '27'},
 {'issue_area': 'Infr

In [57]:
import pandas as pd 
df = pd.DataFrame(data)

In [58]:
import matplotlib.pyplot as plt

In [60]:
Sorted = df.sort_values(['vote_count'], ascending=False)
Sorted.head(3)

,issue_area,issue_path,question,question_path,submission_date,submitter_location,submitters,vote_count
6009,Economy & Jobs,/category/10/,What will you do as president to address the 1...,/questions/6025/vote/,2016-09-28T18:07:00,MO,amie h.,997
6008,Society & Community,/category/5/,What would you do empower girls and women in ...,/questions/16779/vote/,2016-10-07T19:30:00,TX,Milena J.,995
6007,Environment,/category/10/,How will you make the U.S. a leader in solar a...,/questions/220/vote/,2016-09-26T23:31:00,MA,Tina C.,995


In [47]:
df.describe()

,issue_area,issue_path,question,question_path,submission_date,submitter_location,submitters,vote_count
count,6403,6403,6403,6403,6403,6191,6403,6403
unique,10,10,5236,5246,3428,52,3498,995
top,Government & Elections,/category/7/,How to change our fp so others countries pay f...,/questions/5565/vote/,2016-09-27T13:45:00,CA,Ben H.,28
freq,1135,1093,5,4,13,805,45,142


In [53]:
submitter = df.groupby('submitters')
submitter.


In [ ]:
submitter.sort(columns='vote_count').head()

In [21]:
plt.figure()
#df['N'].hist(by=df['Letter'])
df.ix[5]
df['vote_count'].hist(by=df['Letter'])

issue_area                                              Civil Rights
issue_path                                              /category/4/
question           How do we meet international standards for the...
question_path                                  /questions/2796/vote/
submission_date                                  2016-09-27T17:02:00
submitters                                      Philip M. K. from WI
vote_count                                                        75
Name: 5, dtype: object